In [15]:
import os
import sys
bindingspath = os.getcwd() + '/../build/python'
sys.path.append(bindingspath)
print(sys.path)
import pybindings

['/home/adria/anaconda3/lib/python36.zip', '/home/adria/anaconda3/lib/python3.6', '/home/adria/anaconda3/lib/python3.6/lib-dynload', '', '/home/adria/.local/lib/python3.6/site-packages', '/home/adria/anaconda3/lib/python3.6/site-packages', '/home/adria/anaconda3/lib/python3.6/site-packages/IPython/extensions', '/home/adria/.ipython', '/home/adria/Documents/Documents/TUM/dlfs/libdl/python/../build/python']


In [16]:
pybindings.hello("pypy")

0

In [17]:
import numpy as np
aData = np.array([[1, 2], [3, 4]]);
someObject = pybindings.MyClass()
someObject.set_matrix(aData)
bData = someObject.view_matrix()
bData

array([[1., 2.],
       [3., 4.]])

In [6]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)


In [10]:
from sklearn.model_selection import train_test_split
trX, teX, trY, teY = train_test_split(mnist.data / 255.0, mnist.target.astype("int0"), test_size = 0.33)

In [49]:
print("Number of images for training:", trX.shape[0])
print("Values per Image:", trX.shape[1])
print("--- ---")
print("Training Labels:", trY.shape[0])
print("random:", trY[45000])
print("max:", np.max(trY))
print("min:", np.min(trY))

Number of images for training: 46900
Values per Image: 784
--- ---
Training Labels: 46900
random: 4
max: 9
min: 0


In [39]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories='auto')
enc.fit(trY.reshape(-1, 1))
labelsOneHot = enc.transform(trY.reshape(-1, 1)).toarray()


In [48]:
print("Training Labels:", labelsOneHot.shape[0])
print("Training Labels:", labelsOneHot.shape[1])
labelsOneHot[45000,:]

Training Labels: 46900
Training Labels: 10


array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])